In [1]:
import pandas as pd
import numpy as np
import datetime

In [33]:
train = pd.read_csv('data_original/train.csv')
test = pd.read_csv('data_original/test.csv')
test['Count'] = None
frame = [train, test]
commont = pd.concat(frame)
common.Date = common.Date.apply(lambda x: pd.to_datetime(x, format='%d.%m.%Y'))
common = common.sort(['Date'])
common = pd.concat(frame)

/opt/anaconda3/lib/python3.5/site-packages/ipykernel/__main__.py:7: FutureWarning: sort(columns=....) is deprecated, use sort_values(by=.....)


In [8]:
common.to_csv('data_original/common.csv', index=None)
train.to_csv('data_original/train.csv', index=None)
test.to_csv('data_original/test.csv', index=None)



In [31]:
indexes = np.arange(train.shape[0])
np.random.shuffle(indexes)
test_size = int(train.shape[0] * (float(test.shape[0]) / train.shape[0]))

test_indexes = indexes[range(test_size)]
train_indexes = indexes[range(test_size, train.shape[0])]

validate_train = train.iloc[train_indexes]
validate_test = train.iloc[test_indexes]

validate_train.to_csv('data_original/v_train.csv', index=None)
validate_test.to_csv('data_original/v_test.csv', index=None)

In [34]:
v_train = pd.read_csv('data_original/v_train.csv')
v_test = pd.read_csv('data_original/v_test.csv')
v_test['Count'] = None
frame = [v_train, v_test]
v_common = pd.concat(frame)
v_common.Date = v_common.Date.apply(lambda x: pd.to_datetime(x, format='%d.%m.%Y'))
v_common = v_common.sort(['Date'])
v_common = pd.concat(frame)
v_common.to_csv('data_original/v_common.csv', index=None)

/opt/anaconda3/lib/python3.5/site-packages/ipykernel/__main__.py:7: FutureWarning: sort(columns=....) is deprecated, use sort_values(by=.....)


# date.csv

In [35]:
date = pd.read_csv('data_original/v_common.csv')[['Date']]
date.Date = date.Date.apply(lambda x: pd.to_datetime(x))

In [36]:
start_date = date.iloc[0]['Date']
date['Year'] = date.Date.apply(lambda x: x.year)
date['Month'] = date.Date.apply(lambda x: x.month)
date['MonthDay'] = date.Date.apply(lambda x: x.day)
date['WeekDay'] = date.Date.apply(lambda x: x.weekday())

In [37]:
date['YearDay'] = date.Date.apply(lambda x: (x-pd.to_datetime(str(x.year)+'-01-01')).days)

In [38]:
date['ThroughMonth'] = date.Date.apply(lambda x: (x.year - start_date.year) * 12 + x.month - start_date.month)

In [39]:
date['ThroughWeek'] = date.Date.apply(lambda x: int((x - start_date).days / 7))

In [40]:
date['ThroughDay'] = date.Date.apply(lambda x: (x - start_date).days)

In [41]:
date.to_csv('data_transform/date.csv', index=None)

# city.csv


In [36]:
cities = pd.read_csv('data_original/v_common.csv')[['Point']].Point.unique()
city = pd.DataFrame()
city['Point'] = pd.Series(cities)
city['CityName'] = city.Point.apply(lambda x: '-'.join(x.split('-')[0:-1]))
city_names = city.CityName.unique()
city['CityID'] = city.CityName.apply(lambda x: np.where(city_names == x)[0][0]) 
city['BranchNumber'] = city.Point.apply(lambda x: x.split('-')[-1:][0])
# Москва 1 -> Москва
city['CityName'] = city.CityName.apply(lambda x: ('Москва' if x == 'Москва 1' else x))
city.to_csv('data_transform/city.csv', index=None)

# internal_features.csv

In [42]:
src = pd.read_csv('data_transform/date.csv')
src[['Count', 'Point']] = pd.read_csv('data_original/v_common.csv')[['Count', 'Point']]
dst = pd.DataFrame() 

## Mean

### by date independed from points

In [43]:
def calc_mean(src, group_column, target_column):
    mean = src.groupby([group_column])[target_column].mean()
    return src[group_column].apply(lambda x: mean[x])

In [44]:
need_mean_list = src.columns.difference(['Date', 'Count', 'Point'])
for column in need_mean_list:
    dst['MeanFor' + column] = calc_mean(src, column, 'Count')

### by date depended from points

In [45]:
def map_group_two_key(group, x, y):
    return group[x][y]

def calc_mean_by_columns(src, group_columns, target_column):
    mean = src.groupby(group_columns)[target_column].mean()
    return src[group_columns].apply(lambda x: map_group_two_key(mean, *x), axis=1)

In [46]:
need_mean_list = src.columns.difference(['Date', 'Count', 'Point'])
for column in need_mean_list:
    dst['MeanFor' + column + 'ByPoints'] = calc_mean_by_columns(src, [column, 'Point'], 'Count')

In [47]:
dst.to_csv('data_transform/internal_features.csv', index=None)
dst

,MeanForMonth,MeanForMonthDay,MeanForThroughDay,MeanForThroughMonth,MeanForThroughWeek,MeanForWeekDay,MeanForYear,MeanForYearDay,MeanForMonthByPoints,MeanForMonthDayByPoints,MeanForThroughDayByPoints,MeanForThroughMonthByPoints,MeanForThroughWeekByPoints,MeanForWeekDayByPoints,MeanForYearByPoints,MeanForYearDayByPoints
0,100.021576,101.664766,94.523256,100.499432,103.187575,93.714260,111.173982,85.940397,127.000000,154.100000,113.0,127.000000,127.000000,146.588235,162.675000,113.000000
1,104.318793,111.876238,69.904762,96.732438,99.255973,119.683645,99.025944,81.147887,62.720930,62.742857,38.0,60.130435,61.166667,79.486486,70.560311,56.666667
2,108.547069,99.882353,103.123596,114.154833,112.764151,95.637238,111.173982,95.526316,91.369565,82.750000,99.0,119.434783,113.000000,82.040268,124.364486,74.000000
3,104.666796,118.105578,125.111111,106.714286,111.734940,129.730065,106.002184,102.292683,84.166667,113.964286,104.0,78.200000,88.400000,113.073394,77.657371,104.000000
4,106.180925,93.864465,69.039216,96.894737,88.294574,119.683645,99.025944,84.349398,108.814815,68.888889,41.0,45.250000,45.250000,122.068182,59.816327,41.000000
5,100.402677,109.426094,81.700000,103.831571,107.141975,97.309132,111.173982,77.406780,51.833333,70.500000,56.0,51.833333,59.666667,62.029412,68.940860,56.000000
6,106.180925,104.121984,100.055556,110.604361,136.197531,95.637238,106.002184,118.900662,147.126761,131.000000,109.0,126.952381,140.833333,129.496241,121.814672,100.500000
7,111.195173,111.876238,137.629213,118.731380,130.190058,97.309132,111.173982,123.406061,130.560000,135.631579,243.0,151.411765,173.750000,121.933333,149.906103,171.000000
8,104.318793,111.936389,127.358974,113.176676,102.811111,116.270869,111.173982,116.164286,75.512821,67.777778,108.0,98.000000,76.250000,85.555556,94.056338,77.500000
9,106.180925,111.936389,96.945946,110.250455,110.509346,97.309132,111.173982,93.014925,62.300000,67.700000,44.0,62.300000,76.750000,60.631579,68.622047,44.000000


# All in one

In [50]:
data = pd.read_csv('data_original/v_common.csv')[['Point', 'Count']]
date = pd.read_csv('data_transform/date.csv')
city = pd.read_csv('data_transform/city.csv')
internal_features = pd.read_csv('data_transform/internal_features.csv')

need_columns = date.columns
data[need_columns] = date[need_columns]

need_columns = internal_features.columns
data[need_columns] = internal_features[need_columns]

In [51]:
data.to_csv('data_transform/all.csv', index=None)